In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import datetime
import json
import requests

In [11]:
url = 'https://raw.githubusercontent.com/onychinus/FakeNewsNetworkDataset/main/data/whois_label.csv'
data = pd.read_csv(url)
data.head()

,URL,abuse_email,abuse_phone,admin_name,admin_organization,admin_email,admin_address,admin_phone,admin_fax,billing_name,...,technical_address,technical_phone,technical_fax,created_date,updated_date,expires_date,status,nameservers,domain_name,label
0,https://infact.press/2019/10/post-3413/,registrar-abuse@netowl.jp,tel:+81.752568553,REDACTED FOR PRIVACY,XSERVER Inc.,REDACTED FOR PRIVACY,"Osaka, Japan",REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,...,"Osaka, Japan",REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,2019-11-18 11:53:33+00:00,2022-10-22 03:17:46+00:00,2023-11-18 23:59:59+00:00,['client transfer prohibited'],"['ns1.xserver.jp', 'ns2.xserver.jp', 'ns3.xser...",infact.press,Inaccurate
1,https://infact.press/2020/06/post-6452/,registrar-abuse@netowl.jp,tel:+81.752568553,REDACTED FOR PRIVACY,XSERVER Inc.,REDACTED FOR PRIVACY,"Osaka, Japan",REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,...,"Osaka, Japan",REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,2019-11-18 11:53:33+00:00,2022-10-22 03:17:46+00:00,2023-11-18 23:59:59+00:00,['client transfer prohibited'],"['ns1.xserver.jp', 'ns2.xserver.jp', 'ns3.xser...",infact.press,Pants-on-Fire
2,https://infact.press/2020/10/post-9190/,registrar-abuse@netowl.jp,tel:+81.752568553,REDACTED FOR PRIVACY,XSERVER Inc.,REDACTED FOR PRIVACY,"Osaka, Japan",REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,...,"Osaka, Japan",REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,2019-11-18 11:53:33+00:00,2022-10-22 03:17:46+00:00,2023-11-18 23:59:59+00:00,['client transfer prohibited'],"['ns1.xserver.jp', 'ns2.xserver.jp', 'ns3.xser...",infact.press,Unknown-Evidence
3,https://infact.press/2020/11/post-9692/,registrar-abuse@netowl.jp,tel:+81.752568553,REDACTED FOR PRIVACY,XSERVER Inc.,REDACTED FOR PRIVACY,"Osaka, Japan",REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,...,"Osaka, Japan",REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,2019-11-18 11:53:33+00:00,2022-10-22 03:17:46+00:00,2023-11-18 23:59:59+00:00,['client transfer prohibited'],"['ns1.xserver.jp', 'ns2.xserver.jp', 'ns3.xser...",infact.press,Misleading
4,https://infact.press/2020/10/post-9378/,registrar-abuse@netowl.jp,tel:+81.752568553,REDACTED FOR PRIVACY,XSERVER Inc.,REDACTED FOR PRIVACY,"Osaka, Japan",REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,...,"Osaka, Japan",REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,2019-11-18 11:53:33+00:00,2022-10-22 03:17:46+00:00,2023-11-18 23:59:59+00:00,['client transfer prohibited'],"['ns1.xserver.jp', 'ns2.xserver.jp', 'ns3.xser...",infact.press,Inaccurate


In [12]:
data['label'] = data['label'].apply(lambda x: 1 if x != 'Fake' else 0)
data.head()

,URL,abuse_email,abuse_phone,admin_name,admin_organization,admin_email,admin_address,admin_phone,admin_fax,billing_name,...,technical_address,technical_phone,technical_fax,created_date,updated_date,expires_date,status,nameservers,domain_name,label
0,https://infact.press/2019/10/post-3413/,registrar-abuse@netowl.jp,tel:+81.752568553,REDACTED FOR PRIVACY,XSERVER Inc.,REDACTED FOR PRIVACY,"Osaka, Japan",REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,...,"Osaka, Japan",REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,2019-11-18 11:53:33+00:00,2022-10-22 03:17:46+00:00,2023-11-18 23:59:59+00:00,['client transfer prohibited'],"['ns1.xserver.jp', 'ns2.xserver.jp', 'ns3.xser...",infact.press,1
1,https://infact.press/2020/06/post-6452/,registrar-abuse@netowl.jp,tel:+81.752568553,REDACTED FOR PRIVACY,XSERVER Inc.,REDACTED FOR PRIVACY,"Osaka, Japan",REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,...,"Osaka, Japan",REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,2019-11-18 11:53:33+00:00,2022-10-22 03:17:46+00:00,2023-11-18 23:59:59+00:00,['client transfer prohibited'],"['ns1.xserver.jp', 'ns2.xserver.jp', 'ns3.xser...",infact.press,1
2,https://infact.press/2020/10/post-9190/,registrar-abuse@netowl.jp,tel:+81.752568553,REDACTED FOR PRIVACY,XSERVER Inc.,REDACTED FOR PRIVACY,"Osaka, Japan",REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,...,"Osaka, Japan",REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,2019-11-18 11:53:33+00:00,2022-10-22 03:17:46+00:00,2023-11-18 23:59:59+00:00,['client transfer prohibited'],"['ns1.xserver.jp', 'ns2.xserver.jp', 'ns3.xser...",infact.press,1
3,https://infact.press/2020/11/post-9692/,registrar-abuse@netowl.jp,tel:+81.752568553,REDACTED FOR PRIVACY,XSERVER Inc.,REDACTED FOR PRIVACY,"Osaka, Japan",REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,...,"Osaka, Japan",REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,2019-11-18 11:53:33+00:00,2022-10-22 03:17:46+00:00,2023-11-18 23:59:59+00:00,['client transfer prohibited'],"['ns1.xserver.jp', 'ns2.xserver.jp', 'ns3.xser...",infact.press,1
4,https://infact.press/2020/10/post-9378/,registrar-abuse@netowl.jp,tel:+81.752568553,REDACTED FOR PRIVACY,XSERVER Inc.,REDACTED FOR PRIVACY,"Osaka, Japan",REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,...,"Osaka, Japan",REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,2019-11-18 11:53:33+00:00,2022-10-22 03:17:46+00:00,2023-11-18 23:59:59+00:00,['client transfer prohibited'],"['ns1.xserver.jp', 'ns2.xserver.jp', 'ns3.xser...",infact.press,1


In [13]:
data = data.fillna('')
data['created_date'] = pd.to_datetime(data['created_date']).astype(int)/ 10**9
data['updated_date'] = pd.to_datetime(data['updated_date']).astype(int)/ 10**9
data['expires_date'] = pd.to_datetime(data['expires_date']).astype(int)/ 10**9
data.head()

,URL,abuse_email,abuse_phone,admin_name,admin_organization,admin_email,admin_address,admin_phone,admin_fax,billing_name,...,technical_address,technical_phone,technical_fax,created_date,updated_date,expires_date,status,nameservers,domain_name,label
0,https://infact.press/2019/10/post-3413/,registrar-abuse@netowl.jp,tel:+81.752568553,REDACTED FOR PRIVACY,XSERVER Inc.,REDACTED FOR PRIVACY,"Osaka, Japan",REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,...,"Osaka, Japan",REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,1.574078e+09,1.666409e+09,1.700352e+09,['client transfer prohibited'],"['ns1.xserver.jp', 'ns2.xserver.jp', 'ns3.xser...",infact.press,1
1,https://infact.press/2020/06/post-6452/,registrar-abuse@netowl.jp,tel:+81.752568553,REDACTED FOR PRIVACY,XSERVER Inc.,REDACTED FOR PRIVACY,"Osaka, Japan",REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,...,"Osaka, Japan",REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,1.574078e+09,1.666409e+09,1.700352e+09,['client transfer prohibited'],"['ns1.xserver.jp', 'ns2.xserver.jp', 'ns3.xser...",infact.press,1
2,https://infact.press/2020/10/post-9190/,registrar-abuse@netowl.jp,tel:+81.752568553,REDACTED FOR PRIVACY,XSERVER Inc.,REDACTED FOR PRIVACY,"Osaka, Japan",REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,...,"Osaka, Japan",REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,1.574078e+09,1.666409e+09,1.700352e+09,['client transfer prohibited'],"['ns1.xserver.jp', 'ns2.xserver.jp', 'ns3.xser...",infact.press,1
3,https://infact.press/2020/11/post-9692/,registrar-abuse@netowl.jp,tel:+81.752568553,REDACTED FOR PRIVACY,XSERVER Inc.,REDACTED FOR PRIVACY,"Osaka, Japan",REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,...,"Osaka, Japan",REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,1.574078e+09,1.666409e+09,1.700352e+09,['client transfer prohibited'],"['ns1.xserver.jp', 'ns2.xserver.jp', 'ns3.xser...",infact.press,1
4,https://infact.press/2020/10/post-9378/,registrar-abuse@netowl.jp,tel:+81.752568553,REDACTED FOR PRIVACY,XSERVER Inc.,REDACTED FOR PRIVACY,"Osaka, Japan",REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,...,"Osaka, Japan",REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,1.574078e+09,1.666409e+09,1.700352e+09,['client transfer prohibited'],"['ns1.xserver.jp', 'ns2.xserver.jp', 'ns3.xser...",infact.press,1


In [14]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for column in data.columns:
    if data[column].dtype == type(object):
        data[column] = le.fit_transform(data[column])
data.head()

,URL,abuse_email,abuse_phone,admin_name,admin_organization,admin_email,admin_address,admin_phone,admin_fax,billing_name,...,technical_address,technical_phone,technical_fax,created_date,updated_date,expires_date,status,nameservers,domain_name,label
0,12,2,2,1,2,1,2,1,1,1,...,2,1,1,1.574078e+09,1.666409e+09,1.700352e+09,1,2,2,1
1,53,2,2,1,2,1,2,1,1,1,...,2,1,1,1.574078e+09,1.666409e+09,1.700352e+09,1,2,2,1
2,75,2,2,1,2,1,2,1,1,1,...,2,1,1,1.574078e+09,1.666409e+09,1.700352e+09,1,2,2,1
3,80,2,2,1,2,1,2,1,1,1,...,2,1,1,1.574078e+09,1.666409e+09,1.700352e+09,1,2,2,1
4,77,2,2,1,2,1,2,1,1,1,...,2,1,1,1.574078e+09,1.666409e+09,1.700352e+09,1,2,2,1


In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import datetime
import json

X = data.drop('label', axis=1)
y = data['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print('Accuracy:', accuracy_score(y_test, y_pred))

Accuracy: 1.0


In [16]:
def predict_label(json_data):
    df = pd.DataFrame(json_data, index=[0])
    df['created_date'] = pd.to_datetime(df['created_date']).astype(int)/ 10**9
    df['updated_date'] = pd.to_datetime(df['updated_date']).astype(int)/ 10**9
    df['expires_date'] = pd.to_datetime(df['expires_date']).astype(int)/ 10**9
    df['nameservers'] = ','.join(df['nameservers'][0]) if isinstance(df['nameservers'][0], list) else df['nameservers'][0]
    df['status'] = ','.join(df['status'][0]) if isinstance(df['status'][0], list) else df['status'][0]
    for column in df.columns:
        if df[column].dtype == type(object):
            le = LabelEncoder()
            df[column] = df[column].fillna('')
            df[column] = le.fit_transform(df[column])
    df = df.reindex(columns = X.columns, fill_value=0)
    return clf.predict(df)[0]

In [17]:
json_data = {
    'URL': 'https://bitcoin.org/',
    'abuse_email': 'abuse@namecheap.com',
    'abuse_phone': None,
    'admin_address': 'Kalkofnsvegur 2, Reykjavik, Capital Region, 101, IS',
    'admin_email': '43280f540155444088dee67adf69c821.protect@withheldforprivacy.com',
    'admin_fax': None,
    'admin_name': 'Redacted for Privacy',
    'admin_organization': 'Privacy service provided by Withheld for Privacy ehf',
    'admin_phone': None,
    'billing_address': None,
    'billing_email': None,
    'billing_fax': None,
    'billing_name': None,
    'billing_organization': None,
    'billing_phone': None,
    'created_date': '2008-08-18 13:19:55',
    'domain_name': 'bitcoin.org',
    'expires_date': '2029-08-18 13:19:55',
    'nameservers': 'keanu.ns.cloudflare.com',
    'registrant_address': 'Kalkofnsvegur 2, Reykjavik, Capital Region, 101, IS',
    'registrant_email': '43280f540155444088dee67adf69c821.protect@withheldforprivacy.com',
    'registrant_fax': None,
    'registrant_name': 'Redacted for Privacy',
    'registrant_organization': None,
    'registrant_phone': None,
    'registrar_address': '4600 E Washington St #305, Phoenix, Arizona, 85034',
    'registrar_email': 'support@namecheap.com',
    'registrar_fax': None,
    'registrar_name': 'NAMECHEAP INC',
    'registrar_phone': None,
    'status': 'client transfer prohibited',
    'technical_address': 'Kalkofnsvegur 2, Reykjavik, Capital Region, 101, IS',
    'technical_email': '43280f540155444088dee67adf69c821.protect@withheldforprivacy.com',
    'technical_fax': None,
    'technical_name': 'Redacted for Privacy',
    'technical_organization': 'Privacy service provided by Withheld for Privacy ehf',
    'technical_phone': None,
    'updated_date': '2019-11-24 13:58:35'
}
predict_label(json_data)

1

In [19]:
# model save
import pickle
file = "../model/random_forest_model.pkl"
pickle.dump(clf, open(file, "wb"))

In [23]:
# predict the target on the test dataset
def predict(json_data):
    model = pickle.load(open('../model/random_forest_model.pkl','rb'))
    df = pd.DataFrame(json_data, index=[0])
    df['created_date'] = pd.to_datetime(df['created_date']).astype(int)/ 10**9
    df['updated_date'] = pd.to_datetime(df['updated_date']).astype(int)/ 10**9
    df['expires_date'] = pd.to_datetime(df['expires_date']).astype(int)/ 10**9
    df['nameservers'] = ','.join(df['nameservers'][0]) if isinstance(df['nameservers'][0], list) else df['nameservers'][0]
    df['status'] = ','.join(df['status'][0]) if isinstance(df['status'][0], list) else df['status'][0]
    for column in df.columns:
        if df[column].dtype == type(object):
            le = LabelEncoder()
            df[column] = df[column].fillna('')
            df[column] = le.fit_transform(df[column])
    df = df.reindex(columns = X.columns, fill_value=0)
    prediction = model.predict(df)[0]
    return prediction

In [24]:
print(predict(json_data))

1
